In [40]:
import numpy as np
import pandas as pd

tickers = ["BRK","MMC","PGR","ALL","AJG","MET","PRU","TRV","AIG","FNF","MKL","FAF","HIG","BRO","AIZ","AFL","UNM"]
dataframes = []

for ticker in tickers:
    dataframes.append(pd.read_csv("FinancialRatios_"+ticker+".csv"))

for i in range(len(tickers)):
    dataframes[i]['Date'] = pd.to_datetime(dataframes[i]['Date'])
    dataframes[i].set_index('Date', inplace=True)

    

In [41]:
dataframes[16].info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 39 entries, 2014-12-31 to 2024-06-30
Data columns (total 7 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   Working Capital/Total Assets        39 non-null     float64
 1   RetainedEarnings/TotalAssets        39 non-null     float64
 2   EBIT/TotalAssets                    39 non-null     float64
 3   BookValueofEquity/TotalLiabilities  39 non-null     float64
 4   Interest Coverage                   39 non-null     float64
 5   CPI                                 39 non-null     float64
 6   IAK                                 39 non-null     float64
dtypes: float64(7)
memory usage: 2.4 KB


In [42]:
input_columns = dataframes[0].columns
target_columns = [
    "Working Capital/Total Assets",
    "RetainedEarnings/TotalAssets",
    "EBIT/TotalAssets",
    "BookValueofEquity/TotalLiabilities"
]



In [43]:
# Define a function to create sequences
def create_sequences(data, target_indices, seq_length=10):
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data[i:i+seq_length])  # Input: All features
        y.append(data[i+seq_length, target_indices])  # Output: Target variables only
    return np.array(X), np.array(y)

In [65]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
# Train and forecast
forecast_dfs = []
for df in dataframes:
    # Get indices of the target columns
    target_indices = [df.columns.get_loc(col) for col in target_columns]

    # Create sequences for LSTM
    X, y = create_sequences(df.values, target_indices, seq_length=10)

    # Split data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Define the LSTM model
    model = Sequential([
        LSTM(50, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2])),
        Dense(50, activation='relu'),
        Dense(y_train.shape[1])  # Output layer matches the number of target variables
    ])

    # Compile the model with a smaller learning rate
    model.compile(optimizer='adam', loss='mse')

    # Train the model
    history = model.fit(
        X_train, y_train,
        epochs=50,
        batch_size=8,
        validation_data=(X_test, y_test),
        verbose=1
    )

    # Use the most recent sequence for next-step prediction
    last_sequence = df.values[-10:]  # Last 10 steps
    next_step_input = np.expand_dims(last_sequence, axis=0)
    next_step_prediction = model.predict(next_step_input)

    # Create forecast DataFrame
    next_step_date = original_df.index[-1] + pd.DateOffset(months=3)  # Adjust as needed for frequency
    forecast_df = pd.DataFrame({
        "Date": [next_step_date],
        "Working Capital/Total Assets": [next_step_prediction[0, 0]],
        "RetainedEarnings/TotalAsset": [next_step_prediction[0, 1]],
        "EBIT/TotalAssets": [next_step_prediction[0, 2]],
        "BookValueofEquity/TotalLiabilities": [next_step_prediction[0, 3]]
    })
    forecast_df.set_index('Date', inplace=True)
    forecast_dfs.append(forecast_df)
    

Epoch 1/50


C:\Users\alexv\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


3/3 ━━━━━━━━━━━━━━━━━━━━ 5s 383ms/step - loss: 555.5264 - val_loss: 287.2949
Epoch 2/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step - loss: 216.0450 - val_loss: 97.1444
Epoch 3/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - loss: 82.2003 - val_loss: 27.6161
Epoch 4/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step - loss: 40.3254 - val_loss: 17.8991
Epoch 5/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - loss: 36.5949 - val_loss: 13.2909
Epoch 6/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - loss: 18.1752 - val_loss: 10.9381
Epoch 7/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step - loss: 8.4938 - val_loss: 21.7739
Epoch 8/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - loss: 8.8525 - val_loss: 27.9471
Epoch 9/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 9.2349 - val_loss: 21.4586
Epoch 10/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - loss: 6.2719 - val_loss: 12.9245
Epoch 11/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - loss: 4.0817 - val_loss: 7.9988
Epoch 12/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 3.1817 - val_loss:

In [71]:
forecast_dfs[0]

,Working Capital/Total Assets,RetainedEarnings/TotalAsset,EBIT/TotalAssets,BookValueofEquity/TotalLiabilities
Date,,,,
2024-09-30,-1.600211,1.464102,-1.148525,3.625228


In [75]:
dataframes[0].tail()

,Working Capital/Total Assets,RetainedEarnings/TotalAssets,EBIT/TotalAssets,BookValueofEquity/TotalLiabilities,Interest Coverage,CPI,IAK
Date,,,,,,,
2023-06-30,0.073900,0.559292,0.044330,1.126475,37.15,0.030,87.890
2023-09-30,0.090753,0.558641,-0.015649,1.102062,-12.62,0.037,91.650
2023-12-31,0.161844,0.567628,0.044582,1.143351,37.64,0.034,100.125
2024-03-31,0.112058,0.579350,0.015447,1.173763,12.84,0.035,117.390
2024-06-30,0.181206,0.586434,0.035275,1.213784,31.88,0.030,112.920
